# 1. IMPORT LIBRARIES

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Langkah 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. LOAD DATASET

In [5]:
# 2. Load Dataset
file_path = "/content/drive/MyDrive/Colab Notebooks/mobile_game_inapp_purchases.csv"  # ganti sesuai path
df_dirty = pd.read_csv(file_path)

In [6]:
print("=== Dataset Asli ===")
print(df_dirty.head())
print("\nInfo Dataset:")
print(df_dirty.info())
print("\nJumlah Data & Kolom:", df_dirty.shape)

=== Dataset Asli ===
                                 UserID   Age  Gender      Country   Device  \
0  c9889ab0-9cfc-4a75-acd9-5eab1df0015c  49.0    Male       Norway  Android   
1  7c9e413c-ecca-45f2-a780-2826a07952a2  15.0    Male  Switzerland      iOS   
2  fd61e419-1a92-4f43-a8c7-135842ad328a  23.0    Male        China  Android   
3  bdb7f6d1-ff9a-468c-afe7-43f32a94293e  31.0    Male       Mexico  Android   
4  aa7eec14-4846-47b9-b879-9c98038cda04  37.0  Female        India  Android   

       GameGenre  SessionCount  AverageSessionLength SpendingSegment  \
0  Battle Royale             9                 12.83          Minnow   
1     Action RPG            11                 19.39          Minnow   
2       Fighting             9                  8.87          Minnow   
3         Racing            12                 19.56          Minnow   
4  Battle Royale            10                 15.23          Minnow   

   InAppPurchaseAmount  FirstPurchaseDaysAfterInstall PaymentMethod  \


# 3. CEK MISSING VALUES

In [7]:
print("\n=== Cek Missing Values ===")
print(df_dirty.isnull().sum())


=== Cek Missing Values ===
UserID                             0
Age                               60
Gender                            60
Country                           60
Device                            60
GameGenre                         60
SessionCount                       0
AverageSessionLength               0
SpendingSegment                    0
InAppPurchaseAmount              136
FirstPurchaseDaysAfterInstall    136
PaymentMethod                    136
LastPurchaseDate                 136
dtype: int64


# 4. PERBAIKAN MISSING VALUES

In [8]:
# Age → isi dengan median
df_dirty['Age'].fillna(df_dirty['Age'].median(), inplace=True)

# Gender → isi dengan modus
df_dirty['Gender'].fillna(df_dirty['Gender'].mode()[0], inplace=True)

# Country → isi Unknown
df_dirty['Country'].fillna("Unknown", inplace=True)

# Device → isi Unknown
df_dirty['Device'].fillna("Unknown", inplace=True)

# GameGenre → isi Unknown
df_dirty['GameGenre'].fillna("Unknown", inplace=True)

# InAppPurchaseAmount → isi 0 (tidak melakukan pembelian)
df_dirty['InAppPurchaseAmount'].fillna(0, inplace=True)

# FirstPurchaseDaysAfterInstall → isi -1 (belum ada pembelian)
df_dirty['FirstPurchaseDaysAfterInstall'].fillna(-1, inplace=True)

# PaymentMethod → isi "None"
df_dirty['PaymentMethod'].fillna("None", inplace=True)

# LastPurchaseDate → isi "Not Available"
df_dirty['LastPurchaseDate'].fillna("Not Available", inplace=True)

/tmp/ipython-input-1338281712.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dirty['Age'].fillna(df_dirty['Age'].median(), inplace=True)
/tmp/ipython-input-1338281712.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

# 5. Cek ulang Missing Values

In [9]:
print("\n=== Missing Values Setelah Perbaikan ===")
print(df_dirty.isnull().sum())


=== Missing Values Setelah Perbaikan ===
UserID                           0
Age                              0
Gender                           0
Country                          0
Device                           0
GameGenre                        0
SessionCount                     0
AverageSessionLength             0
SpendingSegment                  0
InAppPurchaseAmount              0
FirstPurchaseDaysAfterInstall    0
PaymentMethod                    0
LastPurchaseDate                 0
dtype: int64


# 6. Cek & Hapus Duplikasi

In [10]:
print("\n=== Cek Data Duplikat ===")
dup_count = df_dirty.duplicated().sum()
print(f"Jumlah baris duplikat: {dup_count}")

before = df_dirty.shape[0]
df_clean = df_dirty.drop_duplicates()
after = df_clean.shape[0]


=== Cek Data Duplikat ===
Jumlah baris duplikat: 0


In [11]:
print(f"Jumlah baris sebelum cleaning: {before}")
print(f"Jumlah baris setelah cleaning: {after}")
print(f"Jumlah baris terhapus (duplikat): {before - after}")

Jumlah baris sebelum cleaning: 3024
Jumlah baris setelah cleaning: 3024
Jumlah baris terhapus (duplikat): 0


# 7. Cek Outlier Pada Kolom Age & InAppPurchaseAmount

In [12]:
print("\n=== Range Age ===")
print(df_clean['Age'].describe())

print("\n=== Range InAppPurchaseAmount ===")
print(df_clean['InAppPurchaseAmount'].describe())


=== Range Age ===
count    3024.000000
mean       33.523148
std        11.872885
min        13.000000
25%        24.000000
50%        33.000000
75%        44.000000
max        54.000000
Name: Age, dtype: float64

=== Range InAppPurchaseAmount ===
count    3024.000000
mean       97.969349
std       444.510953
min         0.000000
25%         5.120000
50%        11.455000
75%        17.552500
max      4964.450000
Name: InAppPurchaseAmount, dtype: float64


# 8. Standarisasi Format

In [13]:
# Gender → kapitalisasi awal huruf
df_clean['Gender'] = df_clean['Gender'].str.strip().str.capitalize()

# Country → huruf kapital awal setiap kata
df_clean['Country'] = df_clean['Country'].str.strip().str.title()

# Device → huruf kapital pertama
df_clean['Device'] = df_clean['Device'].str.strip().str.capitalize()

# GameGenre → huruf kecil semua
df_clean['GameGenre'] = df_clean['GameGenre'].str.strip().str.lower()

# PaymentMethod → huruf kapital awal setiap kata
df_clean['PaymentMethod'] = df_clean['PaymentMethod'].str.strip().str.title()

# 9. Cek Nilai Tidak Valid

In [14]:
print("\n=== Cek Nilai Tidak Valid Age ===")
invalid_age = df_clean[(df_clean['Age'] < 5) | (df_clean['Age'] > 100)]
print(invalid_age[['UserID','Age']].head())

print("\n=== Cek Nilai Tidak Valid FirstPurchaseDaysAfterInstall ===")
invalid_days = df_clean[df_clean['FirstPurchaseDaysAfterInstall'] < -1]
print(invalid_days.head())


=== Cek Nilai Tidak Valid Age ===
Empty DataFrame
Columns: [UserID, Age]
Index: []

=== Cek Nilai Tidak Valid FirstPurchaseDaysAfterInstall ===
Empty DataFrame
Columns: [UserID, Age, Gender, Country, Device, GameGenre, SessionCount, AverageSessionLength, SpendingSegment, InAppPurchaseAmount, FirstPurchaseDaysAfterInstall, PaymentMethod, LastPurchaseDate]
Index: []


# 10. Cek Kolom Tidak Relevan

In [15]:
print("\n=== Jumlah nilai unik per kolom ===")
print(df_clean.nunique())

irrelevant_cols = []
single_value_cols = df_clean.nunique()[df_clean.nunique() == 1].index.tolist()
irrelevant_cols.extend(single_value_cols)

df_clean = df_clean.drop(columns=set(irrelevant_cols), errors='ignore')

print("\n=== Kolom dataset sekarang ===")
print(df_clean.columns.tolist())



=== Jumlah nilai unik per kolom ===
UserID                           3024
Age                                42
Gender                              3
Country                            28
Device                              3
GameGenre                          16
SessionCount                       21
AverageSessionLength             1915
SpendingSegment                     3
InAppPurchaseAmount              1839
FirstPurchaseDaysAfterInstall      32
PaymentMethod                       8
LastPurchaseDate                  226
dtype: int64

=== Kolom dataset sekarang ===
['UserID', 'Age', 'Gender', 'Country', 'Device', 'GameGenre', 'SessionCount', 'AverageSessionLength', 'SpendingSegment', 'InAppPurchaseAmount', 'FirstPurchaseDaysAfterInstall', 'PaymentMethod', 'LastPurchaseDate']


# 11. Dataset Setelah Cleansing
Menampilkan dataset akhir setelah proses cleansing:
- Tidak ada missing value penting (semua sudah diisi dengan median, modus, atau kategori default).
- Tidak ada duplikasi data.

- Standarisasi (Kolom Gender → kapitalisasi huruf pertama).
- Standarisasi (Kolom Country → kapitalisasi huruf awal setiap kata).
- Standarisasi (Kolom Device → kapitalisasi huruf pertama).
- Standarisasi (Kolom GameGenre → huruf kecil semua).
- Standarisasi (Kolom PaymentMethod → kapitalisasi huruf awal setiap kata).
- Tidak ada nilai tidak valid pada kolom utama (Age, -  - - FirstPurchaseDaysAfterInstall).
- Tidak ada kolom tidak relevan yang hanya memiliki 1 nilai unik.

In [16]:
print("\n=== Preview Dataset Bersih ===")
display(df_clean.head(10))

print("\n=== Info Dataset Bersih ===")
print(df_clean.info())

print("\n=== Jumlah Baris & Kolom Dataset Bersih ===")
print(df_clean.shape)


=== Preview Dataset Bersih ===


,UserID,Age,Gender,Country,Device,GameGenre,SessionCount,AverageSessionLength,SpendingSegment,InAppPurchaseAmount,FirstPurchaseDaysAfterInstall,PaymentMethod,LastPurchaseDate
0,c9889ab0-9cfc-4a75-acd9-5eab1df0015c,49.0,Male,Norway,Android,battle royale,9,12.83,Minnow,11.40,28.0,Apple Pay,3/19/2025
1,7c9e413c-ecca-45f2-a780-2826a07952a2,15.0,Male,Switzerland,Ios,action rpg,11,19.39,Minnow,6.37,18.0,Debit Card,6/8/2025
2,fd61e419-1a92-4f43-a8c7-135842ad328a,23.0,Male,China,Android,fighting,9,8.87,Minnow,15.81,30.0,Apple Pay,6/2/2025
3,bdb7f6d1-ff9a-468c-afe7-43f32a94293e,31.0,Male,Mexico,Android,racing,12,19.56,Minnow,13.49,9.0,Debit Card,4/1/2025
4,aa7eec14-4846-47b9-b879-9c98038cda04,37.0,Female,India,Android,battle royale,10,15.23,Minnow,10.86,15.0,Paypal,5/5/2025
5,992232c1-a563-4986-aa8b-b70d687d374f,38.0,Female,China,Android,fighting,16,25.97,Dolphin,150.51,28.0,Google Pay,5/5/2025
6,702bafd0-9a08-4269-801e-faf32f9f3ca5,20.0,Female,Italy,Android,simulation,9,10.98,Whale,608.21,24.0,Debit Card,6/4/2025
7,fc470ea9-111f-4637-a9ff-7162d81e2ec3,34.0,Female,Denmark,Ios,unknown,15,5.06,Minnow,19.17,30.0,Gift Card,4/26/2025
8,9b2efc1e-cd4b-48ab-b893-438d57855efa,38.0,Male,Japan,Android,action rpg,10,21.82,Minnow,11.52,1.0,Paypal,4/8/2025
9,25723b60-ea5b-4aff-9f23-97fdf5e98c58,46.0,Male,Russia,Ios,battle royale,18,16.36,Minnow,7.76,19.0,Google Pay,2/3/2025



=== Info Dataset Bersih ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3024 entries, 0 to 3023
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   UserID                         3024 non-null   object 
 1   Age                            3024 non-null   float64
 2   Gender                         3024 non-null   object 
 3   Country                        3024 non-null   object 
 4   Device                         3024 non-null   object 
 5   GameGenre                      3024 non-null   object 
 6   SessionCount                   3024 non-null   int64  
 7   AverageSessionLength           3024 non-null   float64
 8   SpendingSegment                3024 non-null   object 
 9   InAppPurchaseAmount            3024 non-null   float64
 10  FirstPurchaseDaysAfterInstall  3024 non-null   float64
 11  PaymentMethod                  3024 non-null   object 
 12  LastPurchaseDate   

#12 Kesimpulan
- Dataset berhasil dibersihkan dari missing values dengan strategi pengisian nilai default seperti median (Age), modus (Gender), atau kategori khusus seperti Unknown dan None.

- Data duplikat telah dihapus sehingga tidak ada baris ganda yang bisa mengganggu analisis perilaku pemain.
- Beberapa kolom telah dilakukan standarisasi format (misalnya Gender, Country, Device, GameGenre, dan PaymentMethod).

- Nilai yang tidak valid (misalnya usia di luar batas wajar atau jumlah hari pembelian negatif) sudah difilter.

- Kolom yang dianggap tidak relevan (hanya memiliki 1 nilai unik) sudah dihapus agar dataset lebih ringkas.

- Dataset akhir menjadi lebih rapi, konsisten, dan siap digunakan untuk analisis perilaku pembelian dalam game atau tahap data mining selanjutnya.